# Part 1: Baseball Stats Explorations
__This primer illustrates the basics of SQLite usage in Jupyter__

## Boilerplate Setup Code
Run this to set up %sql and %%sql magics

In [3]:
%load_ext sql

## First, a note about Jupyter Magics
Jupyter supports lots of languages, with Python and Markdown being the defaults. However, we can easily mix in code other languages using so-called *magic* extensions. By prepending foreign code with a few *magic* characters (sort of like an incantation) the code that follows is then run, with any outputs translated into Jupyter's native namespace (data structures). So, we can have Python code call R code which calls Bash code ..., all with just a little magic. 

Jupyter magics generally start with the `%` character. A bunch of them are [built into Jupyter](https://ipython.readthedocs.io/en/stable/interactive/magics.html) without any installation required. A few useful ones:
- `%cd`, which changes the directory in which the code that follows is run
- `%load_ext`, which we saw used above to load the %sql and %%sql magics
- `%matplotlb`, which sets up the matplotlib library for interactive use
- `%%bash`, which executes Bash commands just like in a Terminal tab
- `%%html`, which renders HTML code for direct display in the notebook
- `%%js`, which executes Javascript code

You may have noticed that some magics start with `%` while others start with `%%`. 
- The ones with `%` are run "inline" so they can be mixed directly into other code. This way one could for example, have Python code that directly calls SQL code to pull data from a database (without having to set up SqlAlchemy). If the magic code returns an output value, then that value can be captured with a variable. The ony real draw-back is that inline magics have to be on a single line and can's have any line breaks.
- The ones with `%%` are _cell magics_ that apply to a whole cell. Cell magics are always by themselves on the first line of a cell. The code on the lines below are then processed via the magic. If the cell returns a value then it is displayed immediately by Jupyter. However, if we want to capture that value in a variable then we have to use a special [kludge](https://en.wikipedia.org/wiki/Kludge) included in Jupyter for just this purpose. In Jupyter, the special variable `_` (yes, that's an underscore) always contains the output value of the most recently run cell. 

__Here we will be using `%sql` and `%%sql` magics to pull data from a SQLite database.__

## Connecting to the Database
The database we will be using is the `Lahman 2016` data repository that you may have already seen in other classes. We are using it as a sort of common data set that conencts the courses. The database includes [historical baseball data from 1871-2016](https://github.com/jknecht/baseball-archive-sqlite). 

The database tech used is SQLite, a very lightweight package that keeps all data in a single file (sort of like a document) that can be copied and even versioned in GitHub. This has lots of advantages for business analysts. Any time the analysts need data, they can _extract_ it from the DBMS server into a local SQLite database and then _transform_ it in SQLite (or Python or R) prior to analysis. They lose real-time data timeliness, but gain lots of control over way the data is organized.    

In order to connect with any database we have to first provide a *connection string* that specifies:
- what software 'bridge' software is needed
- where to find the database 
- whatever user credentials are needed to access the data inside the database

For a SQLite database, where there is no concept of user accounts or permissions, the connection string is very simple:
```
sqlite:///<path-to-file>
```
The bridge software is `sqlite` and the location of the database is given by `/<path-to-file>`. 

__The code below connects to the `lahman2016.sqlite` database in the same directory as this notebook.__

In [8]:
%sql sqlite:///lahman2016.sqlite

'Connected: @lahman2016.sqlite'

## Using SQL to discover the database design
This can be pretty confusing for SQL newbies. There are no SQL commands for listing all the tables in a database. Instead, we query one or more _metadata_ tables. In SQLite the metadata is in the `sqlite_master` table which describes every data definition object in the database.  Each object has the following fields:
- `type` -- generally either 'table' or 'index'
- `name` -- the object name (which is equal to tbl_name if the object is a table)
- `tbl_name` -- the name of the table that the object describes
- `rootpage` -- the precise location i the b-tree datastructure used to store the data (for SQLite internal use only)
- `sql` -- the SQL DDL code needed to create the object 

In [26]:
%%sql 
/* take a peek at the full sqlite_master table */
SELECT * 
FROM sqlite_master;

 * sqlite:///lahman2016.sqlite
Done.


type,name,tbl_name,rootpage,sql
table,AllstarFull,AllstarFull,2,"CREATE TABLE AllstarFull (playerID TEXT,yearID INTEGER,gameNum INTEGER,gameID TEXT,teamID TEXT,lgID TEXT,GP INTEGER,startingPos INTEGER)"
table,Appearances,Appearances,3,"CREATE TABLE Appearances (yearID INTEGER,teamID TEXT,lgID TEXT,playerID TEXT,G_all INTEGER,GS INTEGER,G_batting INTEGER,G_defense INTEGER,G_p INTEGER,G_c INTEGER,G_1b INTEGER,G_2b INTEGER,G_3b INTEGER,G_ss INTEGER,G_lf INTEGER,G_cf INTEGER,G_rf INTEGER,G_of INTEGER,G_dh INTEGER,G_ph INTEGER,G_pr INTEGER)"
table,AwardsManagers,AwardsManagers,4,"CREATE TABLE AwardsManagers (playerID TEXT,awardID TEXT,yearID INTEGER,lgID TEXT,tie TEXT,notes TEXT)"
table,AwardsPlayers,AwardsPlayers,5,"CREATE TABLE AwardsPlayers (playerID TEXT,awardID TEXT,yearID INTEGER,lgID TEXT,tie TEXT,notes TEXT)"
table,AwardsShareManagers,AwardsShareManagers,6,"CREATE TABLE AwardsShareManagers (awardID TEXT,yearID INTEGER,lgID TEXT,playerID TEXT,pointsWon INTEGER,pointsMax INTEGER,votesFirst INTEGER)"
table,AwardsSharePlayers,AwardsSharePlayers,9,"CREATE TABLE AwardsSharePlayers (awardID TEXT,yearID INTEGER,lgID TEXT,playerID TEXT,pointsWon REAL,pointsMax INTEGER,votesFirst REAL)"
table,Batting,Batting,10,"CREATE TABLE Batting (playerID TEXT,yearID INTEGER,stint INTEGER,teamID TEXT,lgID TEXT,G INTEGER,G_batting INTEGER,AB INTEGER,R INTEGER,H INTEGER,""2B"" INTEGER,""3B"" INTEGER,HR INTEGER,RBI INTEGER,SB INTEGER,CS INTEGER,BB INTEGER,SO INTEGER,IBB INTEGER,HBP INTEGER,SH INTEGER,SF INTEGER,GIDP INTEGER,G_old INTEGER)"
table,BattingPost,BattingPost,11,"CREATE TABLE BattingPost (yearID INTEGER,round TEXT,playerID TEXT,teamID TEXT,lgID TEXT,G INTEGER,AB INTEGER,R INTEGER,H INTEGER,""2B"" INTEGER,""3B"" INTEGER,HR INTEGER,RBI INTEGER,SB INTEGER,CS INTEGER,BB INTEGER,SO INTEGER,IBB INTEGER,HBP INTEGER,SH INTEGER,SF INTEGER,GIDP INTEGER)"
table,CollegePlaying,CollegePlaying,13,"CREATE TABLE CollegePlaying (playerID TEXT,schoolID TEXT,yearID INTEGER)"
table,Fielding,Fielding,14,"CREATE TABLE Fielding (playerID TEXT,yearID INTEGER,stint INTEGER,teamID TEXT,lgID TEXT,POS TEXT,G INTEGER,GS INTEGER,InnOuts INTEGER,PO INTEGER,A INTEGER,E INTEGER,DP INTEGER,PB INTEGER,WP INTEGER,SB INTEGER,CS INTEGER,ZR REAL)"


Most of the time we just want to list the table names, like below.

In [10]:
%%sql
SELECT name
FROM sqlite_master
WHERE type='table'
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


name
AllstarFull
Appearances
AwardsManagers
AwardsPlayers
AwardsShareManagers


Of course, if you want to see (candidate) primary keys then you can do that as well, though it takes a little finagling. Can you take this query apart to see how it works?

In [11]:
%%sql
SELECT tbl_name, SUBSTR(sql, INSTR(sql,'(')) AS candidate_key
FROM sqlite_master
WHERE type='index' and sql like "CREATE UNIQUE%"
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


tbl_name,candidate_key
AllstarFull,"(playerID, yearID, gameNum)"
Appearances,"(yearID, teamID, playerID)"
AwardsManagers,"(yearID, awardID, lgID, playerID)"
AwardsPlayers,"(yearID, awardID, lgID, playerID)"
AwardsShareManagers,"(awardID, yearID, lgID, playerID)"


Tip: Once we have table in mind, we can then also see the CREATE statement that tells us the field definitions.

In [12]:
%%sql
/* the Master table with basic profile data */
SELECT sql 
FROM sqlite_master
WHERE type='table' AND name='Master'
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


sql
"CREATE TABLE Master (playerID TEXT,birthYear INTEGER,birthMonth INTEGER,birthDay INTEGER,birthCountry TEXT,birthState TEXT,birthCity TEXT,deathYear INTEGER,deathMonth INTEGER,deathDay INTEGER,deathCountry TEXT,deathState TEXT,deathCity TEXT,nameFirst TEXT,nameLast TEXT,nameGiven TEXT,weight INTEGER,height INTEGER,bats TEXT,throws TEXT,debut TEXT,finalGame TEXT,retroID TEXT,bbrefID TEXT)"


In [13]:
%%sql
SELECT sql 
FROM sqlite_master
WHERE type='table' AND name='HallOfFame'
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


sql
"CREATE TABLE HallOfFame (playerID TEXT,yearid INTEGER,votedBy TEXT,ballots INTEGER,needed INTEGER,votes INTEGER,inducted TEXT,category TEXT,needed_note TEXT)"


## A couple easy queries
Let's see if we can figure out how this database fits together by exploring a bit.

In [14]:
%%sql 
/* Get a list of all Hall of Fame players */
SELECT nameFirst, nameLast, yearid as induction_year
FROM HallOfFame
    JOIN Master USING (playerID)
WHERE inducted='Y' and category='Player'
ORDER BY yearid
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


nameFirst,nameLast,induction_year
Ty,Cobb,1936
Walter,Johnson,1936
Christy,Mathewson,1936
Babe,Ruth,1936
Honus,Wagner,1936


In [15]:
%%sql
/* add in the number of all star games each HoF player played */
SELECT nameFirst, nameLast, HallOfFame.yearid as induction_year,count(AllstarFull.yearID) AS allstar_appearances
FROM HallOfFame
    JOIN Master USING (playerID)
    LEFT JOIN AllstarFull USING (playerID)
WHERE inducted='Y' and category='Player'
GROUP BY playerID, nameFirst, nameLast
ORDER BY allstar_appearances DESC
LIMIT 5;

 * sqlite:///lahman2016.sqlite
Done.


nameFirst,nameLast,induction_year,allstar_appearances
Hank,Aaron,1982,25
Willie,Mays,1979,24
Stan,Musial,1969,24
Mickey,Mantle,1974,20
Cal,Ripken,2007,19


## (Optional) Python Integration Tips
This section is for those of you using this database in your other classes. If you don't know or care about Python then please skip. However, if you subsequently take a Python class with databases, then perhaps you might like to come back then.   

### You can embed SQL code directly into your Python code without SqlAlchemy

In [21]:
import pandas as pd
 
parks = %sql SELECT parkname, city, state, country FROM Parks LIMIT 5;
parks # parks is a resultset object that is mostly compatible with a dataframe

 * sqlite:///lahman2016.sqlite
Done.


parkname,city,state,country
Candlestick Park,San Francisco,CA,US
AT&T Park,San Francisco,CA,US
Estadio Hiram Bithorn,San Juan,None,PR
Hampden Park Race Track,Springfield,MA,US
Red Stockings Base Ball Park,St. Louis,MO,US


### Converting to pandas DataFrames
To make get a proper dataframe just use the `.DataFrame` method of the resultset

In [24]:
parks_df = parks.DataFrame()
parks_df

,parkname,city,state,country
0,Candlestick Park,San Francisco,CA,US
1,AT&T Park,San Francisco,CA,US
2,Estadio Hiram Bithorn,San Juan,None,PR
3,Hampden Park Race Track,Springfield,MA,US
4,Red Stockings Base Ball Park,St. Louis,MO,US


### Using `%%sql` resultsets
When using `%%sql` the entire cell is SQL code. So, how do we get the results into Python if we can't use Python code in the cell? It's simple, really, just use the `__` pseudo-variable. It always holds the results of the most recently run cell. As long as we take care to have our Python code in the cell immediately after the `%%sql` magic, we should be fine.

In [25]:
%%sql
SELECT parkname, city, state, country
FROM Parks 
LIMIT 10

 * sqlite:///lahman2016.sqlite
Done.


parkname,city,state,country
Candlestick Park,San Francisco,CA,US
AT&T Park,San Francisco,CA,US
Estadio Hiram Bithorn,San Juan,None,PR
Hampden Park Race Track,Springfield,MA,US
Red Stockings Base Ball Park,St. Louis,MO,US
Grand Avenue Park,St. Louis,MO,US
Sportsman's Park I,St. Louis,MO,US
Union Grounds,St. Louis,MO,US
Robison Field,St. Louis,MO,US
Sportsman's Park II,St. Louis,MO,US


In [13]:
parks_df = _.DataFrame()
parks_df

,parkname,city,state,country
0,Candlestick Park,San Francisco,CA,US
1,AT&T Park,San Francisco,CA,US
2,Estadio Hiram Bithorn,San Juan,None,PR
3,Hampden Park Race Track,Springfield,MA,US
4,Red Stockings Base Ball Park,St. Louis,MO,US
5,Grand Avenue Park,St. Louis,MO,US
6,Sportsman's Park I,St. Louis,MO,US
7,Union Grounds,St. Louis,MO,US
8,Robison Field,St. Louis,MO,US
9,Sportsman's Park II,St. Louis,MO,US


### Using multiple databases
Sometimes you will want to *port* data from one database to another. In this case, you will need to connect to both database at the same time. 

The trick is to specify the database connection string for each %sql call. 
```python
%sql <connection-for-db1>
result_db1 = %sql SELECT * FROM ...
%sql <connection-for_db2>
%sql INSERT ... 
```
